In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM


In [59]:
data = pd.read_csv(r"/content/processed_dataset.csv")
print(data.shape)

(20148, 6)


In [67]:
####################Split tHE DATA
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
base_bert_model = AutoModelForMaskedLM.from_pretrained("GroNLP/hateBERT")
train_data = data.sample(frac=0.7, random_state=42)
valid_data = data.sample(frac=0.2, random_state=42)
test_data = data.sample(frac=0.1,random_state=42)
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)
print(data.shape)


Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(14104, 6)
(4030, 6)
(2015, 6)
(20148, 6)


In [61]:
####################
#Get the labels

train_labels = train_data["labels"]
valid_labels = valid_data["labels"]
test_labels = test_data["labels"]

In [62]:

class LLM_HateSpeech_CL(Dataset):
  def __init__(self,texts,labels,tokenizer):


    self.texts = texts
   # self.data = data
    self.labels = labels
    self.tokenizer = tokenizer

    def __len__(self):
      return len(self.texts)


    def __getitem__(self, idx):

       encoding = self.tokenizer(
           self.texts[idx],
           add_special_tokens=True,
           max_length=128

        )


       return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": self.labels[idx]
        }

In [63]:
class LLM_HateSpeech_CL(Dataset):
  def __init__(self,texts,labels,tokenizer):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.texts)


  def __getitem__(self, idx):
     encoding = self.tokenizer(
         self.texts[idx],
         add_special_tokens=True,
         max_length=128

      )


     return {
          "input_ids": encoding["input_ids"].squeeze(0),
          "attention_mask": encoding["attention_mask"].squeeze(0),
          "labels": self.labels[idx]
      }

############################################
#Data loaders



train_dataset = LLM_HateSpeech_CL(train_data["texts"].tolist(),train_labels , tokenizer)
valid_dataset   = LLM_HateSpeech_CL(valid_data["texts"].tolist(), valid_labels, tokenizer)
test_dataset  = LLM_HateSpeech_CL(test_data["texts"].tolist(), test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(valid_dataset, batch_size=16)
test_loader  = DataLoader(test_dataset, batch_size=16)


In [64]:
class HateSpeechClassifier(nn.Module):

  def __init__(self, model):
    super(HateSpeechClassifier, self).__init__()
    self.model = model  # The pre-trained BERT model
    self.classifier = nn.Linear(
        self.model.config.hidden_size,
        2  # Assuming 2 output classes (hate/normal)
    )

  def forward(self, input_ids, attention_mask):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    # Take the [CLS] token embedding (first token's hidden state) as the pooled output
    cls_embedding = outputs.hidden_states[-1][:, 0, :]
    logits = self.classifier(cls_embedding)
    return logits

In [69]:
#################################
#Training loop
def train(model, loss_fn, optimizer, device, epochs):

  # Optimizer should be initialized once outside the epoch loop
  # Assuming optimizer is a class, instantiate it here. For example, if it's torch.optim.AdamW
  optimizer_instance = optimizer(model.parameters(), lr=2e-5)

  for epoch in range(epochs):
    model.train()
    train_loss = 0

    for batch in train_loader:
      input_ids = batch["input_ids"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      labels = batch["labels"].to(device)

      optimizer_instance.zero_grad()
      output = model(input_ids, attention_mask)

      loss = loss_fn(output, labels)
      loss.backward()
      optimizer_instance.step()

      train_loss += loss.item()

    average_train_loss = train_loss / len(train_loader)

    model.eval()
    with torch.no_grad(): # Use no_grad for evaluation
      valid_loss = 0
      correct = 0
      total = 0

      for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device) # Corrected from 'attr_label'

        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels) # Corrected from 'loss_function'
        valid_loss += loss.item() # Accumulate validation loss

        preds = torch.argmax(logits, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

      avg_val_loss = valid_loss / len(val_loader)
      val_accuracy = correct / total

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {average_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

In [70]:
model = HateSpeechClassifier(model=base_bert_model) # Pass the pre-trained model instance
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW # Pass the optimizer class, not an instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

train(model, loss_fn, optimizer, device, 10)

KeyError: 9503